# Manipulation of the diffraction image and the effect onto the real image
### + some code explanations

##### Johannes Bahrenberg, 28.11.2023

--------------------------

###### Mathematik der Fouriertrafo des Einzelspalts

Das Beugungsbild $F(k_y)$ ist die _Fouriertransformierte_ des Spalts $f(y)$:

$F(k_y) = \int_{-\infty}^{\infty}{dy f(y) e}^{-ik_yy} = ... = d \frac{\sin(k_y d/2)}{k_y d/2}$

Das vollständige Spaltbild erhält man wieder durch _Rücktransformation_:

$f(y) = \frac{1}{2 \pi} \int_{-\infty}^{\infty} dk_y F(k_y) e^{ik_y y} = \frac{1}{2 \pi} \int_{-\infty}^{\infty} dk_y F(k_y) [\cos(k_y y) + i \sin(k_y y)] = \frac{1}{2 \pi} \int_{-\infty}^{\infty} dk_y F(k_y) \cos(k_y y) + \text{Im(..)}$

Wegen _Symmetrie_ kann das Integral auch im Intervall $[0, \infty]$ ausgewertet werden. Faktor $2$ dann nicht vergessen! Mit Einsetzen von $F(k_y)$ ergibt sich:

$f(y) = \frac{d}{\pi} \int_{0}^{\infty} dk_y \frac{\sin(k_y d/2)}{k_y d/2} \cos(k_y y) + \text{Im(..)} $

Bei Integration im Intervall $[0, k_{y,n}]$ mit $k_{y,n} = 2 \pi n / d$ ergibt sich ein _unvollständiges Spaltbild_. Hier ist $n$ das Minimum in der Beugungsfunktion $F(k_y)$, bis zu dem integriert wird (größte Frequenz, die zugelassen wird).

In [2]:
# Standard libraries
import numpy as np
import matplotlib.pyplot as plt

# Slider for plots
from matplotlib.widgets import Slider

# Integration
from scipy.integrate import quad

# Write audio file
from scipy.io.wavfile import write

In [3]:
def F(k):
    '''Fouriertrafo Einzelspalt.'''
    return D * np.sin(k*D/2) / (k*D/2)

In [4]:
def integrand_back_transform(k, y):    # heißt im Skript 'spalt', aber diesen Namen finde ich anschaulicher
    '''
    Integrand für Rücktrafo nach f(y) für Einzelspalt.
    
    ERKLÄRUNG
    Wichtig für die scipy.quad() Funktion: Der erste Parameter, k, ist
    die Variable, über die wir integrieren bei der Rücktransformation
    nach f(y) (siehe oben). Wir integrieren über k im Intervall [0, x]
    = [0, 2*pi*n/D].
    
    Der zweite Paramter, y, spiegelt den Ort im realen Raum wieder. 
    Diesen halten wir bei der Berechnung des Integrals fest. D.h. wenn
    wir die Spaltfunktion an irgendeinem Ort y_0 berechnen, berechnen 
    wir f(y_0). Das müssen wir für alle Orte y_0 aus [-D, D] machen,
    daher die for-loop unten.
    '''  
    return D/np.pi * np.sin(k*D/2) / (k*D/2) * np.cos(y*k)

In [5]:
def f_mod(n):
    D = 1    # Spaltbreite, HINWEIS: Konstanten schreibt man mit capslock
    y = np.linspace(-1, 1, 200) * D    # Definitionsbereich
    y_length = len(y)
    
    # compute image with frequencies from 0 to k_n = 2*pi*n/D
    f_modifiziert = np.zeros(y_length)    # pre-allocation, HINWEIS: Erläuterung siehe unten
    x = 2 * np.pi * n / D

    for i in range(y_length):
        #
        # ERKLÄRUNG scipy.quad():
        #
        # Argument 1: Die Funktion, über die wir integrieren. 
        # Argument 2 und 3: Integrationsgrenzen
        # Argument 4: Der aktuelle Ort y_0, für den wir f(y_0) berechnen.
        # Hinweis:  y[i] ist einfach nur eine Zahl. Da das Argument als Tupel
        # übergeben werden muss hier, schreiben wir (y[i],). Das ist die 
        # Schreibweise für ein Tupel mit einem Element.
        #
        result, error = quad(integrand_back_transform, 0, x, args = (y[i],))
        f_modifiziert[i] = result**2

    # normalize to 1
    f_modifiziert_norm = f_modifiziert / np.max(f_modifiziert)
    
    return f_modifiziert_norm

In [6]:
%matplotlib qt

#
# Einzelspalt
#

# general setup
D = 1.    # slid width
ninit = 10    # initial minimum taken
y = np.linspace(-1,1,200) * D
kmax = 60
k = np.linspace(-kmax, kmax, 200)
x = 2 * np.pi * ninit / D

fig, ax = plt.subplots(1,2,figsize=(14,5))

# make space for slider
plt.subplots_adjust(bottom=0.25)

# slider position
ax_slider = plt.axes([0.2, 0.1, 0.6, 0.05])    # xstart, ystart, xwidth, ywidth

# slider
slide = Slider(ax_slider, label='#Minima', valmin=1, valmax=10, valinit=ninit, valstep=1)

# plots
ax[0].plot(k, F(k), label=r'$F(k_y)$')
ax[0].plot(k, F(k)**2, label=r'$F^2(k_y) \sim I(x) $')
ax[0].set_xlabel('k')
ax[0].set_ylabel(r'$F$ und $F^2$')
ax[0].hlines(0, -kmax, kmax, colors='0.0', ls='--')
wall_left = ax[0].axvspan(-kmax, -x, alpha=0.2, color='black', label='Blockiertes\nLicht')
wall_right = ax[0].axvspan(x, kmax, alpha=0.2, color='black')
ax[0].set_xlim(-kmax, kmax)
ax[0].legend()

p, = ax[1].plot(y, f_mod(ninit))
ax[1].set_xlabel('y / d')
ax[1].set_ylabel('Intensitaet / b.E.')
ax[1].set_title(f'Modifiziertes Objektbild des Einzelspalts')
ax[1].set_ylim((0,1.1))


def update(val):
    # get current a factor
    current_n = slide.val
    # compute the function and set y data
    function = f_mod(n=current_n)
    current_x = 2 * np.pi * current_n / D
    wall_left.set_xy([[-kmax, 0], [-kmax, 1], [-current_x, 1], [-current_x, 0], [-kmax, 0]])
    wall_right.set_xy([[current_x, 0], [current_x, 1], [kmax, 1], [kmax, 0], [current_x, 0]])
    p.set_ydata(function)
    fig.canvas.draw()
    
# connect slider with update function
slide.on_changed(update)

plt.show()